In [440]:
loucurinha = pd.read_csv("deputados_temas_e_impeachment.csv", delimiter=";")
loucurinha_p = pd.read_csv("partidos_votos_nomes_impeachment.csv", delimiter=",")
def a(x):
    if x == "SIM":
        return "sim"
    if x == "NAO":
        return "não"
    if x == "AUSEN":
        return "não votou"
    if x == "ABST":
        return "abstenção"

loucurinha.IMPEACHMENT = loucurinha.IMPEACHMENT.apply(lambda x: a(x))
loucurinha[["id_dep", "deputado", "partido", "UF", "IMPEACHMENT"]].to_csv("votacoes_selecionadas/impeachmeant_deputados.csv", index=False, encode="utf8")
loucurinha_p[["nome", "Impeachment"]].to_csv("votacoes_selecionadas/impeachmeant_partidos.csv", index=False, encode="utf8")

In [429]:
import numpy as np
def vote_to_int(vote):
    #'sem orientação' and 'liberado' are used only at parties orientation
    vote_to_int_values = {"sim": 1, "não": 0, "liberado": -1, "abstenção":-2, 
                          "obstrução":-3, "art. 17":-4, "sem orientação": -5,
                         "não votou": -10}

    if vote is None or vote is np.nan:
        return -1
    if vote in vote_to_int_values:
        return vote_to_int_values[vote]
    print(type(vote), vote=="não")
    raise Exception("Not found vote {}".format(vote))

voting = pd.read_csv("votacoes_selecionadas.csv")
voting = voting[["id_dep","nome","voto","nome_votacao","partido","uf"]]
voting["voto_int"] = voting["voto"].apply(vote_to_int)

congressman_infos = voting.drop_duplicates("id_dep")[["id_dep", "nome", "partido", "uf"]]

voting_name_long = voting.pivot(index="id_dep", columns="nome_votacao", values="voto").reset_index()
voting_name_long = voting_name_long.fillna("não votou")
voting_name_long = pd.merge(congressman_infos, voting_name_long, on='id_dep')
voting_name_long = voting_name_long.sort_values(by=["partido", "nome"])

voting_int_long = voting.pivot(index="id_dep", columns="nome_votacao", values="voto_int").reset_index()
voting_int_long = voting_int_long.fillna(-1)
voting_int_long.iloc[:,1:-1] = voting_int_long.iloc[:,1:-1].astype(int)
voting_int_long = pd.merge(congressman_infos, voting_int_long, on='id_dep')
voting_int_long = voting_int_long.sort_values(by=["partido", "nome"])

# voting_int_long.to_csv("deputados_votos.csv", index=False)
# voting_name_long.to_csv("deputados_votos_nomes.csv", index=False)

In [373]:
voting = pd.read_csv("orientacao_partido.csv")
voting.rename(columns={"partido": "nome"})
voting.orientacao_partido = voting.orientacao_partido.fillna("sem orientação")
voting["orientacao_partido_int"] = voting["orientacao_partido"].apply(vote_to_int)

voting_int_long = voting.pivot(index="partido", columns="nome_votacao", 
                               values="orientacao_partido_int").reset_index()
voting_int_long = voting_int_long.fillna(-10) #the party didn't exist at the time of voting


voting_name_long = voting.pivot(index="partido", columns="nome_votacao", 
                               values="orientacao_partido").reset_index()
voting_name_long = voting_name_long.fillna("não votou") #the party didn't exist at the time of voting

voting_int_long.to_csv("partidos_votos.csv", index=False)
voting_name_long.to_csv("partidos_votos_nomes.csv", index=False)

In [465]:
parties_impeachment = pd.read_csv("votacoes_selecionadas/impeachmeant_partidos.csv")
parties_impeachment["Impeachment_int"] = parties_impeachment.Impeachment.apply(vote_to_int)

parties_voting_names = pd.read_csv("partidos_votos_nomes.csv")
parties_voting_names = pd.merge(parties_voting_names, 
            parties_impeachment[["partido","Impeachment"]], on="partido", how="left")
parties_voting_names = parties_voting_names.fillna("não votou")

parties_voting_int = pd.read_csv("partidos_votos.csv")
parties_voting_int = pd.merge(parties_voting_int, 
            parties_impeachment[["partido","Impeachment_int"]], on="partido", how="left")
parties_voting_int = parties_voting_int.fillna(-10)
parties_voting_int = parties_voting_int.iloc[:, 1:-1].astype(int)

parties_voting_int.to_csv("partidos_votos_total.csv", index=False)
parties_voting_names.to_csv("partidos_votos_nomes_total.csv", index=False)

In [466]:
impeachment = pd.read_csv("votacoes_selecionadas/impeachmeant_deputados.csv")
def vote_to_int(vote):
    #'sem orientação' and 'liberado' are used only at parties orientation
    vote_to_int_values = {"sim": 1, "não": 0, "liberado": -1, "abstenção":-2, 
                          "obstrução":-3, "art. 17":-4, "sem orientação": -5,
                         "não votou": -10}

    if vote is None or vote is np.nan:
        return -1
    if vote in vote_to_int_values:
        return vote_to_int_values[vote]
    raise Exception("Not found vote {}".format(vote))

impeachment["Impeachment_int"] = impeachment.Impeachment.apply(vote_to_int)

voting_names = pd.read_csv("deputados_votos_nomes.csv")
voting_names = pd.merge(voting_names, 
            impeachment[["id_dep","Impeachment"]], on="id_dep", how="left")
voting_names = voting_names.fillna("não votou")

voting_int = pd.read_csv("deputados_votos.csv")
voting_int = pd.merge(voting_int, 
            impeachment[["id_dep","Impeachment_int"]], on="id_dep", how="left")
voting_int = voting_int.fillna(-10)
voting_int = voting_int.iloc[:, 4:-1].astype(int)

voting_int.to_csv("deputados_votos_total.csv", index=False)
voting_names.to_csv("deputados_votos_nomes_total.csv", index=False)

## Compare data

In [1069]:
import pandas as pd
import json

new_deputados = json.load(open("deputados_votos_total.json", "r"))
old_deputados = json.load(open("../../quemMeRepresenta/dados/deputados_votos_old.json", "r"))

In [1070]:
def treat(object_):
    df = [{"id":str(dep["id_dep"]), "tema": unidecode.unidecode(tema["tema"].strip()), 
           "value": tema["value"], "nome":unidecode.unidecode(dep["nome"].strip().lower())}
     for dep in object_ for tema in dep["temas"]]
    return pd.DataFrame(df)
new = treat(new_deputados)
new = new[~new.tema.isin(map(unidecode.unidecode, [u"Prosseguimento da denuncia contra Temer",
                         u"Terceirização total 2017", "Lei trabalhista"]))]

old = treat(old_deputados)

old.loc[old.id == "5830793", "id"] = "74833"
old.loc[old.id == "5830700", "id"] = "178942"
old.loc[old.id == "5830515", "id"] = "122158"
old.loc[old.id == "5830461", "id"] = "74471"
old.loc[old.id == "5830356", "id"] = "74160"
old.loc[old.id == "5830335", "id"] = "160528"

In [1071]:
all_ = pd.merge(new, old, on=["id", "tema"], how="outer")
diff = all_[all_.value_x != all_.value_y]


In [1072]:
len(all_), len(diff)

(11860, 896)

In [1073]:
diff.loc[diff.value_x.notnull(), "value_x"] = diff[diff.value_x.notnull()]["value_x"].astype(int)
sum_ = diff[diff.value_x.notnull()][["nome_x", "value_x"]].groupby("nome_x").sum().reset_index()
count_ = diff[diff.value_x.notnull()][["nome_x", "value_x"]].groupby("nome_x").count().reset_index()
sum_ = pd.merge(sum_, count_, on="nome_x")
diff = diff[~diff.nome_x.isin(sum_[sum_.value_x_x == -sum_.value_x_y].nome_x)]
len(diff)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


196

In [1074]:
diff = diff[~diff.id.isin(["74862", "178841", "178883", "178842", "181316","152605",
                          "178965", "178927", "160511", "178851", "151965"])] #duplicated in previous file
len(diff)

0

In [1075]:
diff

,id,nome_x,tema,value_x,nome_y,value_y


In [1077]:
import pandas as pd
import json

new_deputados = json.load(open("partidos_votos_total.json", "r"))
old_deputados = json.load(open("../../quemMeRepresenta/dados/partidos_votos_old.json", "r"))
def treat(object_, column_name):
    df = [{"tema": unidecode.unidecode(tema["tema"].strip()), 
           "value": tema["value"], "nome":unidecode.unidecode(dep[column_name].strip().lower())}
     for dep in object_ for tema in dep["temas"]]
    return pd.DataFrame(df)

new = treat(new_deputados, "nome")
old = treat(old_deputados, "nome")
new = new[~new.tema.isin(map(unidecode.unidecode, [u"Prosseguimento da denuncia contra Temer",
                         u"Terceirização total 2017", "Lei trabalhista"]))]


In [1078]:
all_ = pd.merge(new, old, on=["nome", "tema"], how="outer")
diff = all_[all_.value_x != all_.value_y]
len(diff)

26

In [1079]:
all_c = all_[all_.value_x.isnull()].groupby("nome").count().reset_index()
diff_c = diff[diff.value_x.isnull()].groupby("nome").count().reset_index()
sum_ = pd.merge(all_c, diff_c, on="nome")
diff = diff[~diff.nome.isin(sum_[sum_.value_x_x == -sum_.value_x_y].nome)]
len(diff)

all_c = all_[all_.value_y == -10].groupby("nome").count().reset_index()
diff_c = diff[diff.value_y == -10].groupby("nome").count().reset_index()
sum_ = pd.merge(all_c, diff_c, on="nome")
diff = diff[~diff.nome.isin(sum_[sum_.value_x_x == -sum_.value_x_y].nome)]
len(diff)


6

In [1080]:
diff

,nome,tema,value_x,value_y
60,pen,Cobranca de cursos em universidades publicas,1,-5
120,pmdb,Cobranca de cursos em universidades publicas,1,-5
294,prtb,Terceirizacao,-1,-10
407,psl,Infaticidio indigena,1,-10
408,psl,Maioridade 1,1,-10
409,psl,Maioridade 2,1,-10


In [516]:
votacoes = pd.read_csv("votacoes_selecionadas.csv")

In [867]:
votacoes[votacoes.nome.apply(lambda x: "ero Almeida" in x)]
# votacoes[votacoes.id_dep == 160528]

,tipo,num_pro,ano,id_votacao,resumo,data,hora,objetivo,sessao,nome,id_dep,partido,uf,voto,orientacao_partido,orientacao_gov,cunha,nome_votacao
278,PEC,395,2014,1,Aprovada. em primeiro turno. a Proposta de Eme...,21/10/2015,21:23,SUBSTITUTIVO DA COMISSÃO ESPECIAL - PRIMEIRO T...,5704,Cícero Almeida,178841,pmdb,AL,sim,sim,liberado,sim,Cobrança de cursos em universidades públicas
731,PL,4148,2008,2,Aprovada a Subemenda Substitutiva Global ofere...,28/4/2015,20:41,SUBEMENDA SUBSTITUTIVA DA CDEIC,5513,Cícero Almeida,178841,pmdb,AL,sim,sim,sim,sim,Transgênico
1229,PL,4302,1998,4,Aprovados os dispositivos do Substitutivo do S...,22/3/2017,20:24,SUBSTITUTIVO DO SENADO FEDERAL,6148,Cícero Almeida,178841,pmdb,AL,sim,sim,sim,sim,Terceirização total 2017
1702,PL,1057,2007,1,Aprovada a Subemenda Substitutiva Global de Pl...,26/8/2015,19:14,SUBEMENDA SUBSTITUTIVA GLOBAL,5639,Cícero Almeida,178841,pmdb,AL,sim,sim,não,sim,Infaticídio indígena
2640,PEC,171,1993,1,Rejeitado o Substitutivo adotado pela Comissão...,30/6/2015,00:16,SUBSTITUTIVO DA COMISSÃO ESPECIAL,5587,Cícero Almeida,178841,pmdb,AL,sim,sim,não,sim,Maioridade 1
3127,PEC,171,1993,4,Aprovada a Emenda Aglutinativa nº 16. Sim: 323...,1/7/2015,00:03,EMENDA AGLUTINATIVA Nº 16,5590,Cícero Almeida,178841,pmdb,AL,sim,sim,não,sim,Maioridade 2
3614,PEC,182,2007,3,Rejeitado o artigo 1º do Substitutivo. Sim: 21...,26/5/2015,21:49,ART. 1º DO SUBSTITUTIVO,5545,Cícero Almeida,178841,pmdb,AL,sim,não,liberado,sim,Distritao
4092,PEC,182,2007,5,Rejeitada a Emenda Aglutinativa nº 22. Sim: 26...,26/5/2015,23:53,EMENDA AGLUTINATIVA Nº 22,5545,Cícero Almeida,178841,pmdb,AL,não,não,NaN,sim,Financiamento privado para partidos e candidatos
4546,PEC,182,2007,7,Rejeitado o artigo 8º do Substitutivo. Sim: 13...,10/6/2015,19:19,ARTIGO 8º DO SUBSTITUTIVO - VOTO FACULTATIVO,5562,Cícero Almeida,178841,pmdb,AL,não,não,liberado,liberado,Voto Facultativo
5015,PEC,182,2007,14,Aprovada a Emenda Aglutinativa nº 28. Sim: 330...,27/5/2015,21:08,EMENDA AGLUTINATIVA Nº 28,5547,Cícero Almeida,178841,pmdb,AL,sim,sim,NaN,sim,Financiamento privado para partidos


# Temer

In [330]:
temer_deputados = pd.read_csv("votacoes_selecionadas/temer_deputados.csv")

loucurinha = pd.read_csv("deputados.csv", delimiter=",")
import unidecode
temer_deputados["nome"] = temer_deputados["nome"].apply(lambda x: unidecode.unidecode(x.decode("utf8")))
loucurinha["nome"] = loucurinha["nome"].apply(lambda x: unidecode.unidecode(x.decode("utf8")))
a = pd.merge(temer_deputados, loucurinha[["id_dep", "nome"]], how="left", on="nome")
a = a[a.voto.notnull()]

In [333]:
a[["id_dep","nome","voto"]].to_csv("votacoes_selecionadas/temer_deputados1.csv", index=False)

In [793]:
import pandas as pd
import numpy as np

def add_missing_infos(infos_filename, df):
    dep_missing = pd.read_csv(infos_filename)
    df["id_dep"] = df.id_dep.astype(int)
    df = pd.merge(df, dep_missing,on="id_dep", how="left")
    columns = ["nome","partido", "uf"]
    for c in columns:
        df[c+"_x"] = df[c+"_x"].fillna(df[c+"_y"])
        df.drop(c+"_y", axis=1, inplace=True)
        df.rename(columns={c+"_x":c}, inplace=True)
    return df

def vote_to_int(vote):
    #'sem orientação' and 'liberado' are used only at parties orientation
    vote_to_int_values = {"sim": 1, "não": 0, "liberado": -1, "abstenção":-2,
                          "obstrução":-3, "art. 17":-4, "sem orientação": -5, "não votou":None, "-":None}

    if vote is None or vote is np.nan:
        return -1
    if vote in vote_to_int_values:
        return vote_to_int_values[vote]
    raise Exception("Not found vote {}".format(vote))

external_info = [("votacoes_selecionadas/impeachmeant_deputados.csv", "Impeachment"), 
            ("votacoes_selecionadas/temer_deputados.csv", "Arquivamento da denúncia contra Temer")]
voting_names = pd.read_csv("deputados_votos_nomes.csv")
voting_int = pd.read_csv("deputados_votos.csv")

for info in external_info:
    # info =  ("votacoes_selecionadas/impeachmeant_deputados.csv", "Impeachment")
    # info =  ("votacoes_selecionadas/temer_deputados.csv", "Arquivamento da denúncia contra Temer")
    external_vote = pd.read_csv(info[0])
    external_vote["voto_int"] = external_vote["voto"].apply(vote_to_int)

    voting_names = pd.merge(voting_names,
                            external_vote[["id_dep", "voto"]], on="id_dep", how="outer")
    voting_names = voting_names.rename(columns={"voto":info[1]})


    voting_int = pd.merge(voting_int,
                          external_vote[["id_dep", "voto_int"]], on="id_dep", how="outer")
    voting_int = voting_int.rename(columns={"voto_int":info[1]})
    

voting_names = add_missing_infos("deputados_info_missing.csv", voting_names)
voting_int = add_missing_infos("deputados_info_missing.csv", voting_int)
    
voting_names = voting_names.fillna("não votou")
# voting_int = voting_int.fillna(-10)
# voting_int.iloc[:, 4:] = voting_int.iloc[:, 4:].astype(int)



In [794]:
len(external_vote), len(voting_names)

(511, 580)

In [796]:
voting_int[voting_int.id_dep == 66179]

,id_dep,nome,partido,uf,Cobrança de cursos em universidades públicas,Coincidência reeleição,Cota para mulheres legislativo,Distritao,Financiamento privado para partidos,Financiamento privado para partidos e candidatos,...,Tempo Mandato,Terceirização,Terceirização total 2017,Terrorismo,Transgênico,Tributação serviços de internet,Voto Facultativo,Voto impresso,Impeachment,Arquivamento da denúncia contra Temer
22,66179,Norma Ayub,dem,ES,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,1.0


In [735]:
x = pd.merge(voting_names, external_vote[["id_dep", "voto"]], on="id_dep", how="outer")


In [736]:
external_vote

,id_dep,deputado,partido,uf,voto,voto_int
0,178864,Adail Carneiro,phs,CE,sim,1.0
1,178914,Adalberto Cavalcanti,ptb,PE,não,0.0
2,178890,Adelmo Carneiro Leão,pt,MG,não,0.0
3,178968,Adelson Barreto,ptb,SE,sim,1.0
4,178903,Adilton Sachetti,psb,MT,sim,1.0
5,141372,Aelton Freitas,pr,MG,não,0.0
6,160508,Afonso Florence,pt,BA,não,0.0
7,136811,Afonso Hamm,pp,RS,sim,1.0
8,178835,Afonso Motta,pdt,RS,não,0.0
9,160527,Aguinaldo Ribeiro,pp,PB,sim,1.0


In [729]:
votacoes[votacoes.nome.apply(lambda x: "Celso" in x)]
# votacoes[votacoes.id_dep == 160528]

,tipo,num_pro,ano,id_votacao,resumo,data,hora,objetivo,sessao,nome,id_dep,partido,uf,voto,orientacao_partido,orientacao_gov,cunha,nome_votacao
262,PEC,395,2014,1,Aprovada. em primeiro turno. a Proposta de Eme...,21/10/2015,21:23,SUBSTITUTIVO DA COMISSÃO ESPECIAL - PRIMEIRO T...,5704,Celso Jacob,73568,pmdb,RJ,sim,NaN,liberado,sim,Cobrança de cursos em universidades públicas
272,PEC,395,2014,1,Aprovada. em primeiro turno. a Proposta de Eme...,21/10/2015,21:23,SUBSTITUTIVO DA COMISSÃO ESPECIAL - PRIMEIRO T...,5704,Celso Russomanno,73441,prb,SP,sim,sim,liberado,sim,Cobrança de cursos em universidades públicas
303,PEC,395,2014,1,Aprovada. em primeiro turno. a Proposta de Eme...,21/10/2015,21:23,SUBSTITUTIVO DA COMISSÃO ESPECIAL - PRIMEIRO T...,5704,Celso Maldaner,141405,pmdb,SC,sim,NaN,liberado,sim,Cobrança de cursos em universidades públicas
715,PL,4148,2008,2,Aprovada a Subemenda Substitutiva Global ofere...,28/4/2015,20:41,SUBEMENDA SUBSTITUTIVA DA CDEIC,5513,Celso Jacob,73568,pmdb,RJ,sim,sim,sim,sim,Transgênico
725,PL,4148,2008,2,Aprovada a Subemenda Substitutiva Global ofere...,28/4/2015,20:41,SUBEMENDA SUBSTITUTIVA DA CDEIC,5513,Celso Russomanno,73441,prb,SP,sim,sim,sim,sim,Transgênico
758,PL,4148,2008,2,Aprovada a Subemenda Substitutiva Global ofere...,28/4/2015,20:41,SUBEMENDA SUBSTITUTIVA DA CDEIC,5513,Celso Maldaner,141405,pmdb,SC,sim,sim,sim,sim,Transgênico
1206,PL,4302,1998,4,Aprovados os dispositivos do Substitutivo do S...,22/3/2017,20:24,SUBSTITUTIVO DO SENADO FEDERAL,6148,Celso Jacob,73568,pmdb,RJ,abstenção,sim,sim,sim,Terceirização total 2017
1223,PL,4302,1998,4,Aprovados os dispositivos do Substitutivo do S...,22/3/2017,20:24,SUBSTITUTIVO DO SENADO FEDERAL,6148,Celso Russomanno,73441,prb,SP,sim,sim,sim,sim,Terceirização total 2017
1261,PL,4302,1998,4,Aprovados os dispositivos do Substitutivo do S...,22/3/2017,20:24,SUBSTITUTIVO DO SENADO FEDERAL,6148,Celso Maldaner,141405,pmdb,SC,sim,sim,sim,sim,Terceirização total 2017
1315,PL,4302,1998,4,Aprovados os dispositivos do Substitutivo do S...,22/3/2017,20:24,SUBSTITUTIVO DO SENADO FEDERAL,6148,Celso Pansera,72912,pmdb,RJ,não,sim,sim,sim,Terceirização total 2017


In [1002]:
orient[["partido", "nome_votacao", "orientacao_partido"]]

,partido,nome_votacao,orientacao_partido
0,PpPtbPscPhs,Cobrança de cursos em universidades públicas,sim
1,PmdbPen,Cobrança de cursos em universidades públicas,sim
2,PT,Cobrança de cursos em universidades públicas,liberado
3,PSDB,Cobrança de cursos em universidades públicas,sim
4,PrbPtnPmnPrpPsdcPtcPslPtdoB,Cobrança de cursos em universidades públicas,sim
5,PR,Cobrança de cursos em universidades públicas,liberado
6,PSD,Cobrança de cursos em universidades públicas,sim
7,PSB,Cobrança de cursos em universidades públicas,liberado
8,DEM,Cobrança de cursos em universidades públicas,sim
9,PDT,Cobrança de cursos em universidades públicas,sim


In [1000]:
import copy
orient = pd.read_csv("parties_orientation_selected.csv")
coaliations = {
    "PpPtbPscPhs": ["Pp","Ptb","Psc","Phs"],
    "PmdbPen": ["Pmdb","Pen"],
    "PrbPtnPmnPrpPsdcPtcPslPtdoB": ["Prb","Ptn","Pmn","Prp","Psdc","Ptc","Psl","PtdoB"],
    "Solidaried": ["Solidaried"],
    "PCdoB": ["PCdoB"],
    "PmdbPpPtbPscPhsPen": ["Pmdb","Pp","Ptb","Psc","Phs","Pen"],
    "PrbPtnPmnPrpPsdcPrtbPtcPslPtdoB": ["Prb","Ptn","Pmn","Prp","Psdc","Prtb","Ptc","Psl","PtdoB"],
    "PtbProsPsl": ["Ptb","Pros","Psl"],
    "PpPtnPhs...": ["Pp","Ptn","Phs","Prp","Ptdob"],
    "PpPtnPTdoB":["Pp","Ptn","PTdoB"]
}

exclude = {"Maioria", "Minoria", "GOV."}

parties_already_know = pd.read_csv("parties_already_know.csv")
parties_already_know = set(parties_already_know["partido"])

parties_already_know = map(lambda x: x.lower(), parties_already_know)
rows = []
for index, row in orient.iterrows():
    party = row["partido"]
    party = party.replace("Repr.","")
    if party in exclude:
        continue
    if party in coaliations:
        parties_splited = coaliations[party]
        for party_splited in parties_splited:
            row_ = copy.deepcopy(row)
            row_["partido"] = party_splited
            rows.append(row_)
    elif party.lower() in parties_already_know:
        rows.append(row)
    else:
        raise Exception("Party not found: {}".format(party))
df = pd.DataFrame.from_records(rows)
df["partido"] = df["partido"].apply(lambda x: x.upper())
df.to_csv("parties_orientation_selected_parsed.csv", index=False)

In [1020]:
df[df.partido == "PMB"]

,tipo,num_pro,ano,id_votacao,resumo,data,hora,objetivo,sessao,partido,orientacao_partido,nome_votacao
